In [45]:
import pandas as pd
import numpy as np

In [46]:
dengue_data = pd.read_csv("data/cleaned_dengue_merged.csv")
dengue_labels = pd.read_csv("data/dengue_labels_train.csv")
dengue_data

,city,year,weekofyear,total_cases,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,...,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
0,sj,1990,18,4,1990-04-30,0.122600,0.103725,0.198483,0.177617,12.42,...,32.00,73.365714,12.42,14.012857,2.628571,25.442857,6.900000,29.4,20.0,16.0
1,sj,1990,19,5,1990-05-07,0.169900,0.142175,0.162357,0.155486,22.82,...,17.94,77.368571,22.82,15.372857,2.371429,26.714286,6.371429,31.7,22.2,8.6
2,sj,1990,20,4,1990-05-14,0.032250,0.172967,0.157200,0.170843,34.54,...,26.10,82.052857,34.54,16.848571,2.300000,26.714286,6.485714,32.2,22.8,41.4
3,sj,1990,21,3,1990-05-21,0.128633,0.245067,0.227557,0.235886,15.36,...,13.90,80.337143,15.36,16.672857,2.428571,27.471429,6.771429,33.3,23.3,4.0
4,sj,1990,22,6,1990-05-28,0.196200,0.262200,0.251200,0.247340,7.52,...,12.20,80.460000,7.52,17.210000,3.014286,28.942857,9.371429,35.0,23.9,5.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1451,iq,2010,21,5,2010-05-28,0.342750,0.318900,0.256343,0.292514,55.30,...,45.00,88.765714,55.30,18.485714,9.800000,28.633333,11.933333,35.4,22.4,27.0
1452,iq,2010,22,8,2010-06-04,0.160157,0.160371,0.136043,0.225657,86.47,...,207.10,91.600000,86.47,18.070000,7.471429,27.433333,10.500000,34.7,21.7,36.6
1453,iq,2010,23,1,2010-06-11,0.247057,0.146057,0.250357,0.233714,58.94,...,50.60,94.280000,58.94,17.008571,7.500000,24.400000,6.900000,32.2,19.2,7.4
1454,iq,2010,24,1,2010-06-18,0.333914,0.245771,0.278886,0.325486,59.67,...,62.33,94.660000,59.67,16.815714,7.871429,25.433333,8.733333,31.2,21.0,16.0


In [47]:
X_train = np.array(dengue_data)[:,1:]
y_train = np.array(dengue_labels)[:,-1]
X_train[:,3], y_train

(array(['1990-04-30', '1990-05-07', '1990-05-14', ..., '2010-06-11',
        '2010-06-18', '2010-06-25'], dtype=object),
 array([4, 5, 4, ..., 1, 1, 4], dtype=object))

In [48]:
def split_date(date_str):
    year, month, day = map(int, date_str.split('-'))
    return year, month, day

dates = X_train[:, 3]

if type(dates[0]) == str:
    years, months, days = zip(*map(split_date, dates))

    # Replace the date column with the split components
    X_train[:, 3] = years
    X_train = np.insert(X_train, 3, months, axis=1)
    X_train = np.insert(X_train, 4, days, axis=1)

# Now convert the numerical columns to float
numerical_columns = [2, 3, 4]
X_train[:, numerical_columns] = X_train[:, numerical_columns].astype(float)

In [49]:
X_train[:,3], X_train[:,4], X_train[:,5], X_train[:,6]

(array([4.0, 5.0, 5.0, ..., 6.0, 6.0, 6.0], dtype=object),
 array([30.0, 7.0, 14.0, ..., 11.0, 18.0, 25.0], dtype=object),
 array([1990, 1990, 1990, ..., 2010, 2010, 2010], dtype=object),
 array([0.1226, 0.1699, 0.03225, ..., 0.2470571, 0.3339143, 0.2981857],
       dtype=object))

In [50]:
X_train[0,5]

1990

In [51]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
y_train_scaled = scaler.fit_transform(y_train.reshape(-1, 1))  # Reshape y_train before scaling

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
y_train = y_train.reshape(-1,1)
y_test = y_test.reshape(-1,1)
# Print the shapes of training and testing data
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)


(1164, 26) (1164, 1)
(292, 26) (292, 1)


In [52]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Define the number of features in your dataset
num_features = 26  # Adjust this based on your actual dataset

# Initialize the model
model = Sequential()

# Add input layer
model.add(Dense(64, activation='softplus', input_shape=(num_features,)))

# Add hidden layers
model.add(Dense(32, activation='softplus'))
model.add(Dense(16, activation='softplus'))

# Add output layer
model.add(Dense(1, activation='linear'))

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])  # Mean Squared Error loss for regression task

# Print model summary
model.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 64)                1728      
                                                                 
 dense_13 (Dense)            (None, 32)                2080      
                                                                 
 dense_14 (Dense)            (None, 16)                528       
                                                                 
 dense_15 (Dense)            (None, 1)                 17        
                                                                 
Total params: 4,353
Trainable params: 4,353
Non-trainable params: 0
_________________________________________________________________


In [53]:
print(X_train.shape, y_train.shape)
history = model.fit(X_train.astype(float), y_train.astype(float), epochs=100, batch_size=32)

(1164, 26) (1164, 1)
Epoch 1/100


37/37 [==============================] - 0s 715us/step - loss: 24099.9961 - mae: 98.5983
Epoch 2/100
37/37 [==============================] - 0s 659us/step - loss: 1284.2712 - mae: 19.8244
Epoch 3/100
37/37 [==============================] - 0s 682us/step - loss: 1139.0029 - mae: 18.0489
Epoch 4/100
37/37 [==============================] - 0s 705us/step - loss: 1013.1793 - mae: 17.2519
Epoch 5/100
37/37 [==============================] - 0s 685us/step - loss: 880.8043 - mae: 15.8010
Epoch 6/100
37/37 [==============================] - 0s 702us/step - loss: 725.5252 - mae: 14.1502
Epoch 7/100
37/37 [==============================] - 0s 722us/step - loss: 580.0128 - mae: 13.2127
Epoch 8/100
37/37 [==============================] - 0s 747us/step - loss: 440.8921 - mae: 11.1251
Epoch 9/100
37/37 [==============================] - 0s 732us/step - loss: 293.3674 - mae: 8.7710
Epoch 10/100
37/37 [==============================] - 0s 768us/step - loss: 141.5691 - mae: 6.2239
Epoch 11/100
37/37

In [54]:
prediction = model.predict(X_test.astype(float))
prediction[20], y_test[20]

10/10 [==============================] - 0s 515us/step


(array([12.268897], dtype=float32), array([11], dtype=object))

In [55]:
from sklearn.metrics import mean_absolute_error

# Assuming y_pred and y_test are the predicted and true values respectively
mae = mean_absolute_error(y_test, prediction)

print("Mean Absolute Error:", mae)

from sklearn.metrics import r2_score

# Assuming y_true and y_pred are the true and predicted target values, respectively
r2 = r2_score(y_test, prediction)

print("R-squared (R2) Score:", r2)



Mean Absolute Error: 1.6100099919593498
R-squared (R2) Score: 0.999098706599801


In [56]:
# now to get weather characteristics from city year and week
weather_to_characteristics = np.array(dengue_labels)[:X_train.shape[0],:]
weather_to_characteristics

array([['sj', 1990, 18, 4],
       ['sj', 1990, 19, 5],
       ['sj', 1990, 20, 4],
       ...,
       ['iq', 2004, 44, 6],
       ['iq', 2004, 45, 22],
       ['iq', 2004, 46, 37]], dtype=object)

In [62]:
from sklearn.preprocessing import OneHotEncoder

# Assuming 'labels' is a list or array containing the string labels 'sj' and 'iq'
labels = ['sj', 'iq']

# Create the OneHotEncoder object
encoder = OneHotEncoder()

# Fit the encoder to the labels and transform them
encoded_labels = encoder.fit_transform(np.array(weather_to_characteristics[:,0]).reshape(-1, 1))

# Convert the encoded labels to an array
encoded_labels_array = encoded_labels.toarray()

print("Encoded labels:\n", encoded_labels_array)

Encoded labels:
 [[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]


In [64]:
weather_to_characteristics = np.delete(weather_to_characteristics, 0, axis=1)
print(weather_to_characteristics)
weather_to_characteristics = np.concatenate((encoded_labels_array, weather_to_characteristics), axis=1)
weather_to_characteristics = np.delete(weather_to_characteristics, 4, axis=1)
weather_to_characteristics.shape

[[0.0 1.0 1990]
 [0.0 1.0 1990]
 [0.0 1.0 1990]
 ...
 [1.0 0.0 2004]
 [1.0 0.0 2004]
 [1.0 0.0 2004]]


(1164, 4)

In [65]:
weather_to_characteristics_y = X_train[:, 6:]
weather_to_characteristics_y.shape

(1164, 20)

In [71]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Define the number of features in your dataset
num_features = 4  # Adjust this based on your actual dataset

# Initialize the model
model_1 = Sequential()

# Add input layer
model_1.add(Dense(128, activation='softplus', input_shape=(num_features,)))

# Add hidden layers
model_1.add(Dense(64, activation='softplus'))
model_1.add(Dense(32, activation='softplus'))

# Add output layer
model_1.add(Dense(20, activation='linear'))

# Compile the model
model_1.compile(optimizer='adam', loss='mse', metrics=['mae'])  # Mean Squared Error loss for regression task

# Print model summary
model_1.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_20 (Dense)            (None, 128)               640       
                                                                 
 dense_21 (Dense)            (None, 64)                8256      
                                                                 
 dense_22 (Dense)            (None, 32)                2080      
                                                                 
 dense_23 (Dense)            (None, 20)                660       
                                                                 
Total params: 11,636
Trainable params: 11,636
Non-trainable params: 0
_________________________________________________________________


In [72]:
history = model.fit(weather_to_characteristics.astype(float), weather_to_characteristics_y.astype(float), epochs=100, batch_size=32)

Epoch 1/100
37/37 [==============================] - 0s 838us/step - loss: 23053.7266 - mae: 91.6024
Epoch 2/100
37/37 [==============================] - 0s 840us/step - loss: 18064.9941 - mae: 79.7824
Epoch 3/100
37/37 [==============================] - 0s 895us/step - loss: 5029.2925 - mae: 42.2720
Epoch 4/100
37/37 [==============================] - 0s 964us/step - loss: 640.4763 - mae: 14.1388
Epoch 5/100
37/37 [==============================] - 0s 997us/step - loss: 384.7398 - mae: 8.1799
Epoch 6/100
37/37 [==============================] - 0s 890us/step - loss: 377.9340 - mae: 7.4965
Epoch 7/100
37/37 [==============================] - 0s 903us/step - loss: 378.1604 - mae: 7.5939
Epoch 8/100
37/37 [==============================] - 0s 864us/step - loss: 377.6323 - mae: 7.5207
Epoch 9/100
37/37 [==============================] - 0s 858us/step - loss: 379.0117 - mae: 7.5585
Epoch 10/100
37/37 [==============================] - 0s 909us/step - loss: 378.3803 - mae: 7.5974
Epoch 11/1

In [70]:
y=model_1.predict(weather_to_characteristics.astype(float))
print(y.shape, weather_to_characteristics_y.shape)
from sklearn.metrics import mean_absolute_error

# Assuming y_pred and y_test are the predicted and true values respectively
mae = mean_absolute_error(y, weather_to_characteristics_y)

print("Mean Absolute Error:", mae)

from sklearn.metrics import r2_score

# Assuming y_true and y_pred are the true and predicted target values, respectively
r2 = r2_score(y, weather_to_characteristics_y)

print("R-squared (R2) Score:", r2)


37/37 [==============================] - 0s 473us/step
(1164, 1) (1164, 20)


ValueError: y_true and y_pred have different number of output (1!=20)